Library by Library Collection Dates

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

In [2]:
# makes list of file paths
file_paths = [f'librarycollection/collection_{i}.csv' for i in range(1, 26)]

# Reads data from each file into a DataFrame
dataframes = [pd.read_csv(file) for file in file_paths]

# Concatenate all DataFrames into one
library_collection = pd.concat(dataframes, ignore_index=True)

In [3]:
def cleaning_data(library_collection):
    library_collection = library_collection.dropna(subset = ["ItemCollection", "ItemLocation"])
    library_collection = library_collection.drop(['BibNum', 'Title','Author','ISBN','ReportDate','ObjectId'], axis=1)
    library_collection = library_collection[~library_collection["ItemCollection"].isin(["Music", "Listening Device", 
                                                                                        "Children's Music", "Children's DVD", 
                                                                                        "Laptop", "Adult DVD", "Interlibrary Loan", "NaN",
                                                                                        "Electronic", "Adult Blu-Ray", "Children's Blu-ray",
                                                                                        "Digital Camera","Telescope","Hotspot","Board Game", "Launchpad"])]
    
    return library_collection
librarycleaner = cleaning_data(library_collection)

In [4]:
def combining_collections(librarycleaner):
    """
    This function combines ItemCollection into adult, teen, and children. 
    It also removes collections that cannot be sorted by age. The categories and sorting are:
Adult = Adult Non-Fiction, Adult Fiction, Adult Paperback, Adult Audiobook, Adult Reference, 
    Adult Paperbacks Tall, Mystery, Kentucky History, Large Print, Science Fiction, International Collection, 
    Magazines and Newspaper, Government Documents, Urban Fiction, Western
Teen = Teen Non-Fiction, Older Teen Fiction, Younger Teen  Fiction, Younger Teen  Audiobook, 
    OlderTeen Audiobook, Younger Teen  Paperback, Older Teen Paperback, Teen Non-Fiction Audiobook, 
    College Shop, Teen Reference  
Children = Children's Non-Fiction, Children's Picture Book, Preschool Picture Book, Children's Fiction, 
    Children's Paperback, Children's Easy Reader, Children's Board Book, Children's Easy Reader Paperback, 
    Children's Picture Paperback, Children's Audiobook, Storytime Collection, Readalongs, Children's Magazine, 
    Children's Reference, Children's Non-Fiction Paperback  
Unsorted/removed = Caldecott/Newbery, Oversize, ELL Collection, Bestsellers, Natural Resources, 
    Book Discussion Kit, Big Book, Telereference, Holiday

The second part of the function simplifies the locations of the items.
    Remote Shelving - Main is combined with Main
    Childrens Main Library is combined with Main
    Main Teen is combined with Main
    Remote Shelving - Shawnee is combined with Shawnee
    Childrens Bookmobile, Adult Bookmobile, and Content Management are removed as they have no physical location
    """
    librarycleaner = librarycleaner[~librarycleaner["ItemCollection"].isin(["Caldecott/Newbery", "Oversize", "ELL Collection",
                                                                            "Bestsellers", "Natural Resources", 
                                                                            "Book Discussion Kit", "Big Book", "Telereference", "Holiday"])]
    librarycleaner["ItemCollection"] = librarycleaner["ItemCollection"].replace(["Adult Non-Fiction", "Adult Fiction", "Adult Paperback", "Adult Audiobook", 
                                                                                 "Adult Reference", "Adult Paperbacks Tall", "Mystery", "Kentucky History", 
                                                                                 "Large Print", "Science Fiction", "International Collection", 
                                                                                 "Magazines and Newspaper", "Government Documents", "Urban Fiction", 
                                                                                 "Western"], "Adult")
    librarycleaner["ItemCollection"] = librarycleaner["ItemCollection"].replace(["Teen Non-Fiction", "Older Teen Fiction", "Younger Teen  Fiction", 
                                                                                 "Younger Teen  Audiobook", "OlderTeen Audiobook", "Younger Teen  Paperback", 
                                                                                 "Older Teen Paperback", "Teen Non-Fiction Audiobook", "College Shop",
                                                                                 "Teen Reference"], "Teen")
    librarycleaner["ItemCollection"] = librarycleaner["ItemCollection"].replace(["Children's Non-Fiction", "Children's Picture Book", "Preschool  Picture Book", 
                                                                                 "Children's Fiction", "Children's Paperback", "Children's Easy Reader", 
                                                                                 "Children's Board Book", "Children's Easy Reader Paperback", "Children's Picture Paperback", 
                                                                                 "Children's Audiobook", "Storytime Collection", "Readalongs", "Children's Magazine", 
                                                                                 "Children's Reference", "Children's Non-Fiction Paperback"], "Children")
    librarycleaner = librarycleaner[~librarycleaner["ItemLocation"].isin(["Childrens Bookmobile", "Adult Bookmobile", "Content Management"])]
    librarycleaner["ItemLocation"] = librarycleaner["ItemLocation"].replace(["Remote Shelving - Main", "Childrens Main Library", "Main Teen"], "Main")
    librarycleaner["ItemLocation"] = librarycleaner["ItemLocation"].replace("Remote Shelving - Shawnee", "Shawnee")
    return(librarycleaner)
librarysorted = combining_collections(librarycleaner)

C:\Users\Nick Campbell\AppData\Local\Temp\ipykernel_6044\166721550.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  librarycleaner["ItemCollection"] = librarycleaner["ItemCollection"].replace(["Adult Non-Fiction", "Adult Fiction", "Adult Paperback", "Adult Audiobook",
C:\Users\Nick Campbell\AppData\Local\Temp\ipykernel_6044\166721550.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  librarycleaner["ItemCollection"] = librarycleaner["ItemCollection"].replace(["Teen Non-Fiction", "Older Teen Fiction", 

In [6]:
replacements = {'Northeast': 'NORTHEAST REGIONAL', 'South Central': 'SOUTH CENTRAL REGIONAL', 'Southwest': 'SOUTHWEST BRANCH',
                'Main': 'MAIN', 'Portland': 'PORTLAND', 'Iroquois': 'IROQUOIS', 
                'Highlands - Shelby Park': 'HIGHLANDS-SHELBY PARK', 'Bon Air': 'BON AIR REGIONAL',
                'St Matthews': 'ST. MATTHEWS/ELINE', 'Western': 'WESTERN', 'Shawnee': 'SHAWNEE', 'Crescent Hill': 'CRESCENT HILL',
                'Jeffersontown': 'JEFFERSONTOWN', 'Fairdale': 'FAIRDALE', 'Middletown': 'MIDDLETOWN', 'Newburg': 'NEWBURG',
                'Shively': 'SHIVELY'}
librarysorted2 = librarysorted.replace(replacements)

In [11]:
librarysortedadult = librarysorted2[~librarysorted.ItemCollection.str.contains("Teen", "Children")]
library_by_year = librarysortedadult[['PublicationYear','ItemLocation']].groupby(['ItemLocation']).value_counts(normalize=True).reset_index(name='Percent')
library_by_year['Percent'] = library_by_year['Percent'] * 100
library_by_year1 = library_by_year[library_by_year.PublicationYear != 0]

In [8]:
northeast = library_by_year1.loc[library_by_year1['ItemLocation'] == 'NORTHEAST REGIONAL']
southeast = library_by_year1.loc[library_by_year1['ItemLocation'] == 'SOUTH CENTRAL REGIONAL']
southwest = library_by_year1.loc[library_by_year1['ItemLocation'] == 'SOUTHWEST BRANCH']
main = library_by_year1.loc[library_by_year1['ItemLocation'] == 'MAIN']
portland = library_by_year1.loc[library_by_year1['ItemLocation'] == 'PORTLAND']
iroquois = library_by_year1.loc[library_by_year1['ItemLocation'] == 'IROQUOIS']
highlands = library_by_year1.loc[library_by_year1['ItemLocation'] == 'HIGHLANDS-SHELBY PARK']
bonair = library_by_year1.loc[library_by_year1['ItemLocation'] == 'BON AIR REGIONAL']
matthews = library_by_year1.loc[library_by_year1['ItemLocation'] == 'ST. MATTHEWS/ELINE']
western = library_by_year1.loc[library_by_year1['ItemLocation'] == 'WESTERN']
shawnee = library_by_year1.loc[library_by_year1['ItemLocation'] == 'SHAWNEE']
crescent = library_by_year1.loc[library_by_year1['ItemLocation'] == 'CRESCENT HILL']
jtown = library_by_year1.loc[library_by_year1['ItemLocation'] == 'JEFFERSONTOWN']
fairdale = library_by_year1.loc[library_by_year1['ItemLocation'] == 'FAIRDALE']
middletown = library_by_year1.loc[library_by_year1['ItemLocation'] == 'MIDDLETOWN']
newburg = library_by_year1.loc[library_by_year1['ItemLocation'] == 'NEWBURG']
shively = library_by_year1.loc[library_by_year1['ItemLocation'] == 'SHIVELY']


In [13]:

def year_divisions(x):
    x_1 = x[x['PublicationYear'].isin([2020, 2021, 2022, 2023, 2024, 2025])]
    x['2020-2025'] = x_1['Percent'].sum()
    x_2 = x[x['PublicationYear'].isin([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])]
    x['2010-2019'] = x_2['Percent'].sum()
    x_3 = x[x['PublicationYear'].isin([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009])]
    x['2000-2009'] = x_3['Percent'].sum()
    x_4 = x[x['PublicationYear'].isin([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999])]
    x['1990-1999'] = x_4['Percent'].sum()
    x_5 = x[x['PublicationYear'].isin([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989])]
    x['1980-1989'] = x_5['Percent'].sum()
    x_6 = x[x['PublicationYear'].isin([1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979])]
    x['1970-1979'] = x_6['Percent'].sum()
    x_7 = x[x['PublicationYear'].isin([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 
                                       1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969])]
    x['1950-1969'] = x_7['Percent'].sum()
    x_8  = x[x['PublicationYear'].isin([1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 
                                        1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 
                                        1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 
                                        1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940,
                                        1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949])]
    x['1900-1949'] = x_6['Percent'].sum()
    x.drop(['PublicationYear', 'Percent'], axis=1, inplace=True)
    x.drop(x.index[1:], inplace=True)
    return(x)

north_east = year_divisions(northeast)
south_east = year_divisions(southeast)
south_west = year_divisions(southwest)
main_main = year_divisions(main)
port_land = year_divisions(portland)
iro_quois = year_divisions(iroquois)
high_lands = year_divisions(highlands)
bon_air = year_divisions(bonair)
matt_hews = year_divisions(matthews)
west_ern = year_divisions(western)
shaw_nee = year_divisions(shawnee)
cres_cent = year_divisions(crescent)
j_town = year_divisions(jtown)
fair_dale = year_divisions(fairdale)
middle_town = year_divisions(middletown)
new_burg = year_divisions(newburg)
shive_ly = year_divisions(shively)


In [14]:
library_yearly = pd.concat([north_east, south_east, south_west, main_main, port_land, iro_quois, high_lands, 
                            bon_air, matt_hews, west_ern, shaw_nee, cres_cent, j_town, fair_dale, middle_town,
                            new_burg, shive_ly])

In [16]:
map_dict = {'BON AIR REGIONAL':'40220', 'CRESCENT HILL': '40206', 'FAIRDALE': '40118', 'HIGHLANDS-SHELBY PARK': '40204', 
            'IROQUOIS': '40215', 'JEFFERSONTOWN': '40299', 'MAIN': '40203', 'MIDDLETOWN': '40243', 'NEWBURG': '40218', 
            'NORTHEAST REGIONAL': '40223', 'PORTLAND': '40212', 'SHAWNEE': '40211', 'SHIVELY': '40216', 'SOUTH CENTRAL REGIONAL': '40219', 
            'SOUTHWEST BRANCH': '40272', 'ST. MATTHEWS/ELINE': '40207', 'WESTERN': '40203'}

library_yearly["zipcode"] = library_yearly["ItemLocation"].map(map_dict)             

In [17]:
library_yearly

,ItemLocation,2020-2025,2010-2019,2000-2009,1990-1999,1980-1989,1970-1979,1950-1969,1900-1949,zipcode
1129,NORTHEAST REGIONAL,10.893533,70.857252,12.230478,3.709318,1.104181,0.469670,0.399261,0.469670,40223
1489,SOUTH CENTRAL REGIONAL,10.091105,64.334984,18.630828,4.529263,1.111363,0.452591,0.383026,0.452591,40219
1596,SOUTHWEST BRANCH,11.430699,57.082463,22.406121,5.830452,1.398350,0.739397,0.520316,0.739397,40272
651,MAIN,4.076713,28.012570,27.660572,15.449019,6.424605,5.455824,4.531079,5.455824,40203
1239,PORTLAND,27.455507,52.639809,12.830442,3.015861,1.198898,1.079753,0.997840,1.079753,40212
435,IROQUOIS,13.187176,52.948715,22.589665,7.000408,1.728256,0.891314,0.613628,0.891314,40215
321,HIGHLANDS-SHELBY PARK,20.843661,53.218874,15.783968,5.366686,1.605458,1.059693,0.757248,1.059693,40204
0,BON AIR REGIONAL,13.143706,54.593970,23.265570,5.738336,1.378191,0.741362,0.525418,0.741362,40220
1716,ST. MATTHEWS/ELINE,19.540034,55.221526,17.302745,5.004762,1.321905,0.639632,0.474749,0.639632,40207
1832,WESTERN,20.545282,36.814036,12.400937,8.503215,3.389324,7.476449,6.115735,7.476449,40203


In [18]:
pd.set_option('display.max_rows', None)